In [19]:
import nbaPlayerStatModeling as nbaModel

In [20]:
nba = nbaModel.playerStatModel(
        day_offset = 0, 
        season = '2024-25', 
        perMode = 'PerGame'
)

In [21]:
nba.get_teams_playing()


3  games today...


In [22]:
nba.get_teams_data(
    base_url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType={stattype}&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
)

In [23]:
nba.df_games_long

,GAME_ID,GAME_DATE_EST,GAME_STATUS_TEXT,TEAM_TYPE,TEAM_ID,oppTeamId,OREB_PCT,DREB_PCT,PACE,oppAst,oppPts
0,0022400752,2025-02-09,NaN,Home,1610612765,1610612766,0.308,0.726,99.56,25.5,113.5
1,0022400752,2025-02-09,NaN,Away,1610612766,1610612765,0.317,0.713,98.16,26.4,112.3
2,0022400753,2025-02-09,NaN,Home,1610612745,1610612761,0.363,0.703,98.71,22.6,108.7
3,0022400753,2025-02-09,NaN,Away,1610612761,1610612745,0.307,0.690,99.95,26.2,116.5
4,0022400754,2025-02-09,NaN,Home,1610612749,1610612755,0.242,0.726,100.15,25.9,112.9
5,0022400754,2025-02-09,NaN,Away,1610612755,1610612749,0.268,0.686,97.39,27.6,113.3


In [24]:
nba.get_players_playing(
    url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight=',
    minute_cutoff = 0
)

102 players returned from 6 teams, out of a total 6


In [25]:
nba.get_props()

333 prop bets for 35 players...


In [26]:
nba.df_players = nba.df_players.iloc[[1,10,19,25,30], :]
nba.get_player_home_adv(
    use_default=False

)

1 / 5 player splits..
2 / 5 player splits..
3 / 5 player splits..
4 / 5 player splits..
5 / 5 player splits..


In [27]:
nba.model_expected_minutes()
nba.calculate_model_inputs()

In [69]:
nba.df_players = nba.df_players.fillna(0)

In [75]:
nba.model_expected_stats_poisson()

In [76]:
nba.df_players

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,shotshareFG2A,shotshareFG3A,shotshareFTA,expReb,expAst,expPts,REBoOdds,ASToOdds,REBoProb,ASToProb
0,1630595,Cade Cunningham,1610612765,DET,35.7,9.5,21.1,0.452,2.3,6.4,...,0.565385,0.246154,0.188462,7.362742,10.442623,25.262834,-289.0,-147.0,0.743207,0.596086
1,202331,Paul George,1610612755,PHI,31.9,6.1,14.2,0.430,2.4,6.5,...,0.461078,0.389222,0.149701,6.828345,5.090772,19.259470,-209.0,-296.0,0.676834,0.747486
2,1628415,Dillon Brooks,1610612745,HOU,32.5,4.9,11.9,0.415,2.3,6.1,...,0.426471,0.448529,0.125000,4.402719,1.205333,12.550923,-178.0,194.0,0.641026,0.339299
3,1629018,Gary Trent Jr.,1610612749,MIL,24.9,3.5,8.2,0.431,2.3,5.5,...,0.300000,0.611111,0.088889,3.117286,1.371242,9.624349,-151.0,151.0,0.602567,0.398199
4,1642266,Ja'Kobe Walter,1610612761,TOR,19.5,2.9,7.0,0.406,1.1,3.4,...,0.423529,0.400000,0.176471,2.683143,1.410790,10.210140,-1363.0,-309.0,0.931652,0.756049


In [74]:
nba.df_players[['PLAYER_NAME', 'OREB', 'DREB', 'REB', 'reb_line', 'expReb']]

,PLAYER_NAME,OREB,DREB,REB,reb_line,expReb
0,Cade Cunningham,0.9,5.5,6.4,5.5,7.362742
1,Paul George,0.6,4.7,5.3,5.5,6.828345
2,Dillon Brooks,1.2,2.6,3.9,3.5,4.402719
3,Gary Trent Jr.,0.2,2.2,2.4,2.5,3.117286
4,Ja'Kobe Walter,0.7,1.7,2.4,0.0,2.683143


In [56]:
for i in nba.df_players.columns: print(i)

PLAYER_ID
PLAYER_NAME
TEAM_ID
TEAM_ABBREVIATION
MIN
FGM
FGA
FG_PCT
FG3M
FG3A
FG3_PCT
FTM
FTA
FT_PCT
OREB
DREB
REB
AST
TOV
STL
BLK
BLKA
PF
PFD
PTS
PLUS_MINUS
homeTeam
oppTeamId
PACE
OREB_PCT
DREB_PCT
oppPACE
oppPts
oppAst
PACEadj
OREBadj
DREBadj
PTSadj
ASTadj
threes_line
stl_line
reb_line
ra_line
pts_line
pra_line
pr_line
pa_line
blk_line
ast_line
sb_line
threes_oOdds
stl_oOdds
reb_oOdds
ra_oOdds
pts_oOdds
pra_oOdds
pr_oOdds
pa_oOdds
blk_oOdds
ast_oOdds
sb_oOdds
threes_uOdds
stl_uOdds
reb_uOdds
ra_uOdds
pts_uOdds
pra_uOdds
pr_uOdds
pa_uOdds
blk_uOdds
ast_uOdds
sb_uOdds
MIN_HOMEadj
FGM_HOMEadj
FGA_HOMEadj
FG3M_HOMEadj
FG3A_HOMEadj
FTM_HOMEadj
FTA_HOMEadj
OREB_HOMEadj
DREB_HOMEadj
REB_HOMEadj
AST_HOMEadj
TOV_HOMEadj
STL_HOMEadj
BLK_HOMEadj
BLKA_HOMEadj
PTS_HOMEadj
MIN_ROADadj
FGM_ROADadj
FGA_ROADadj
FG3M_ROADadj
FG3A_ROADadj
FTM_ROADadj
FTA_ROADadj
OREB_ROADadj
DREB_ROADadj
REB_ROADadj
AST_ROADadj
TOV_ROADadj
STL_ROADadj
BLK_ROADadj
BLKA_ROADadj
PTS_ROADadj
MINadj
FG2M
FG2A
FG2_PCT
FGA_FT

In [30]:
nba.df_players[['PLAYER_NAME','homeTeam', 'MINadj','PACEadj', 
                'OREB', 'OREBadj', 'OREB_HOMEadj',  'DREB', 'DREBadj','DREB_HOMEadj', 'expReb', 
                'AST', 'ASTadj', 'expAst', 
                'PTS', 'PTSadj', 'expPts']]

,PLAYER_NAME,homeTeam,MINadj,PACEadj,OREB,OREBadj,OREB_HOMEadj,DREB,DREBadj,DREB_HOMEadj,expReb,AST,ASTadj,expAst,PTS,PTSadj,expPts
0,Cade Cunningham,1,1,1.007106,0.9,0.982652,0.535714,5.5,0.965917,1.286957,7.362742,9.5,0.999748,10.442623,25.5,0.991145,25.262834
1,Paul George,0,1,0.986124,0.6,0.938142,0.857143,4.7,1.071984,1.391304,6.828345,4.5,0.980813,5.090772,16.6,0.996440,19.259470
2,Dillon Brooks,1,1,0.993778,1.2,1.061402,1.251553,2.6,0.980059,1.113043,4.402719,1.5,0.992174,1.205333,13.6,1.028213,12.550923
3,Gary Trent Jr.,1,1,1.014071,0.2,1.075097,1.254545,2.2,1.035214,1.231313,3.117286,1.1,1.045191,1.371242,10.2,0.999971,9.624349
4,Ja'Kobe Walter,0,1,1.006261,0.7,1.016891,1.578947,1.7,0.900863,1.118421,2.683143,1.4,0.855844,1.410790,7.8,0.959372,10.210140
